## First Import the library

In [ ]:
import pycaret
import pandas as pd

## create dataframe

In [ ]:
train_dir = 'C:/Users/ILuz/Videos/DE/archive_5/train.csv'
test_dir = 'C:/Users/ILuz/Videos/DE/archive_5/test.csv'
train = pd.read_csv(train_dir)



In [ ]:
test = pd.read_csv(test_dir)

In [ ]:
train.head()

## address logitude and latitude is useless for training so i'm going to drop it.

In [ ]:
train.drop(['ADDRESS','LONGITUDE','LATITUDE'],axis=1,inplace=True)
train.head()

## check 5 num summaries

In [ ]:
train.describe()

## something isn't right let's chack with boxplot

In [ ]:
boxplot = train.boxplot(column=['SQUARE_FT']) 
boxplot

## as you can see there is a outlier in our dataset

In [ ]:
train['TARGET(PRICE_IN_LACS)'].sort_values(ascending=False).head()

## use iqr to identify the outliers and save it to the variable and drop it.

In [ ]:


Q1 = train['TARGET(PRICE_IN_LACS)'].quantile(0.25)
Q3 = train['TARGET(PRICE_IN_LACS)'].quantile(0.75)
IQR = Q3 - Q1


outliers = train[((train['TARGET(PRICE_IN_LACS)'] < (Q1 - 1.5 * IQR)) |(train['TARGET(PRICE_IN_LACS)'] > (Q3 + 1.5 * IQR)))]


In [ ]:
train.drop(outliers.index,inplace=True)

In [ ]:
train.describe()

In [ ]:
train.shape

In [ ]:
train.drop_duplicates(inplace=True)

## get dummy for categorical columns.

In [ ]:
train.head()
train=pd.get_dummies(train,columns=["POSTED_BY"])

## map number to categorical columns.

In [ ]:
mapping = {'BHK': 0, 'RK': 1}
train['BHK_OR_RK'] = train['BHK_OR_RK'].map(mapping)
train.head()

## setup our model using pycaret abd compare to find best model for dataset

In [ ]:
from pycaret.regression import RegressionExperiment
s = RegressionExperiment()
s.setup(train,target='TARGET(PRICE_IN_LACS)')
compare = s.compare_models()

In [ ]:
train.columns

## prepare test data for fitting in a model.

In [ ]:
test.drop(['ADDRESS','LONGITUDE','LATITUDE'],axis=1,inplace=True)
test=pd.get_dummies(test,columns=["POSTED_BY"])
test['BHK_OR_RK'] = test['BHK_OR_RK'].map(mapping)

In [ ]:
predictions = s.predict_model(compare, data=test)
predictions.head()

## save our final results.

In [ ]:
predictions.to_csv('targetpred.csv')